In [58]:
using Gtk
using Gtk.ShortNames
gtk = Gtk

Gtk

In [136]:
function enterANOVAData(rows,cols,sampleSize,rowNames,colNames,table)
    #=This won't be very efficient but i'm in a hurry=#
    grid = Grid()
    for i in 1:rows
       label = Label(rowNames[i]) 
       grid[1,i+1] =  label
    end
    
    for j in 1:cols
        label = Label(colNames[j])
        grid[j+1,1]= label
    end
    
    #=In each block we want a grid of size sampleSize=#
    
    for i in 2:rows+1
        for j in 2:cols+1
            gr = Grid()
            for k in 1:sampleSize
                a = Entry()
                setproperty!(a,:text,"")
                gr[1,k] = a
            end
            fr = Frame(gr)
            grid[j,i] = fr
            println("filled ", i, "and ",j)
        end
    end
    
    setproperty!(grid, :column_homogeneous, true)
    scrWin = ScrolledWindow()
    setproperty!(scrWin, :min_content_width, 400)
    setproperty!(scrWin, :min_content_height, 400)
    
    push!(scrWin,grid)
    window = Window()
    vBox = Box(:v)
    
    #=Generate array button=#
    button = Button("Generate Array")
    
     #=Associate the button with the function generate array =#
    id = signal_connect(button, "clicked") do widget
     return generateArray(grid,rows,cols,sampleSize,table)    
    end
    
    push!(vBox,scrWin)
    push!(vBox,button)
    push!(window,vBox)
    
    showall(window)
end

function generateArray(grid, rows, cols,sampleSize,table)
    for i in 2:rows+1
        row = []
        for j in 2:cols+1
            col = []
            for k in 1:sampleSize
                num = 0
                for child in grid[j,i]
                num = get(tryparse(Float64,getproperty(child[1,k],:text,String)))
                println(num)
                push!(col,num)
                end
            end
            push!(row,col)
        end
        push!(table,row)
    end
    println("Array generated!")
end
    

generateArray (generic function with 1 method)

In [137]:
#=Vamos a hacer las funciones que calculen todo lo que pide la hamster en el ejercicio 3=#
function meanGenerator(data,dim)
    a = size(data,1)
    b = size(data[1],1)
    n = size(data[1][1],1)
    if(dim ==1)
        result = []
        for j in 1:b
            arr =[]
            for k in 1:n
                sum = 0
                for i in 1:a
                    sum+=data[i][j][k]
                end
                sum = sum/a
                push!(arr,sum)
            end
            push!(result,arr)
        end
        return result
        
        
    elseif(dim ==2)
        result = []
            for i in 1:a
            arr =[]
            for k in 1:n
                sum = 0
                for j in 1:b
                    sum+=data[i][j][k]
                end
                sum = sum/b
                push!(arr,sum)
            end
            push!(result,arr)
        end
    return result
        
    elseif(dim == 3)
        result = []
        for i in 1:a
            arr =[]
            for j in 1:b
                sum = 0
                for k in 1:n
                    sum+=data[i][j][k]
                end
                sum = sum/n
                push!(arr,sum)
            end
            push!(result,arr)
        end
        return result
        
    elseif(dim == 4)
        #=Here get the total mean=#
        numData = 0
        result = 0
        for i in 1:a
            for j in 1:b
                for k in 1:n
                    numData+=1
                    result+=data[i][j][k]
                end
            end
        end
        return result/(a*b*n)
    end
end

function SCT(data)
    gMean = meanGenerator(data,4)
    a = size(data,1)
    b = size(data[1],1)
    n = size(data[1][1],1)
    sum = 0
    for i in 1:a
        for j in 1:b
            for k in 1:n
                sum+=(data[i][j][k]-gMean)^2
            end
        end
    end
    return sum 
end

function SCTr(data)
    means = meanGenerator(data,3)
    gMean = meanGenerator(data,4)
    a = size(data,1)
    b = size(data[1],1)
    n = size(data[1][1],1)
    sum = 0
    for i in 1:a
        for j in 1:b
            for k in 1:n
            sum+=(means[i][j]-gMean)^2
            end
        end
    end
    return sum 
end

function SCResidual(data)
    return SCT(data)-SCTr(data)
end

function SCA(data)
    gMean = meanGenerator(data,4)
    aMean = meanGeneratorDual(data,2,3)
    sum = 0
    a = size(data,1)
    b = size(data[1],1)
    n = size(data[1][1],1)
    for i in 1:a
        for j in 1:b
            for k in 1:n
            sum+=(aMean[i]-gMean)^2
            end
        end
    end
    return sum
end

function SCB(data)
    gMean = meanGenerator(data,4)
    bMean = meanGeneratorDual(data,1,3)
    sum = 0
    a = size(data,1)
    b = size(data[1],1)
    n = size(data[1][1],1)
    for i in 1:a
        for j in 1:b
            for k in 1:n
                sum+=(bMean[j]-gMean)^2
            end
        end
    end
    return sum
end

function meanGeneratorDual(data,dim1,dim2)
    if(dim1 ==2 && dim2 == 3)
        mean2 = meanGenerator(data,3)
        a = size(mean2,1)
        b = size(mean2[1],1)
        result = []
        for i in 1:a
            sum = 0
            for j in 1:b
                sum+=mean2[i][j]
            end
            sum = sum/b
            push!(result,sum)
        end
        return result
        
        elseif(dim1 == 1 && dim2 == 3)
        mean1 = meanGenerator(data,3)
        a = size(mean1,1)
        b = size(mean1[1],1)
        result = []
        for j in 1:b
            sum = 0
            for i in 1:a
                sum+=mean1[i][j]
            end
            sum = sum/a
            push!(result,sum)
        end
        return result
    end
end

function SCAB(data)
   return SCTr(data)-SCA(data)-SCB(data) 
end

function CMA(data)
   a = size(data,1)
   return SCA(data)/(a-1) 
end

function CMB(data)
   b = size(data[1],1)
   return SCB(data)/(b-1) 
end

function CMAB(data)
   a = size(data,1)
   b = size(data[1],1)
   return SCAB(data)/((a-1)*(b-1)) 
end

function CMRES(data)
    a = size(data,1)
    b = size(data[1],1)
    n = size(data[1][1],1)
   return (SCResidual(data))/((a*b)*(n-1)) 
    
end

function RVA(data)
    return CMA(data)/CMRES(data)
end

function RVB(data)
    return CMB(data)/CMRES(data)    
end

function RVAB(data)
    return CMAB(data)/CMRES(data)    
end


RVAB (generic function with 1 method)

In [155]:
function ANOVATable(table)
    grid = Grid()
    #=Encabezados=#
    grid[1,1] = Label("Fuente")
    grid[2,1] = Label("SC")
    grid[3,1] = Label("Grados de libertad")
    grid[4,1] = Label("CM")
    grid[5,1] = Label("R.V.")
    
    #=Fuentes=#
    grid[1,2] = Label("A")
    grid[1,3] = Label("B")
    grid[1,4] = Label("AB")
    grid[1,5] = Label("Tratamientos")
    grid[1,6] = Label("Residual")
    grid[1,7] = Label("Total")
    
    #=Llenamos SC=#
    grid[2,2] = Label(string(SCA(table)))
    grid[2,3] = Label(string(SCB(table)))
    grid[2,4] = Label(string(SCAB(table)))
    grid[2,5] = Label(string(SCTr(table)))
    grid[2,6] = Label(string(SCResidual(table)))
    grid[2,7] = Label(string(SCA(table)+SCB(table)+SCAB(table)+SCTr(table)+SCResidual(table)))
    
    #=Los grados de libertad=#
    a = size(table,1)
    b = size(table[1],1)
    n = size(table[1][1],1)
    
    grid[3,2] = Label(string(a-1))
    grid[3,3] = Label(string(b-1))
    grid[3,4] = Label(string((a-1)*(b-1)))
    grid[3,5] = Label(string(a*b -1))
    grid[3,6] = Label(string(a*b*(n-1)))
    grid[3,7] = Label(string(a*b*n -1))
    
    #=Los CM=#
    grid[4,2] = Label(string(CMA(table)))
    grid[4,3] = Label(string(CMB(table)))
    grid[4,4] = Label(string(CMAB(table)))
    grid[4,5] = Label("")
    grid[4,6] = Label(string(CMRES(table)))
    grid[4,7] = Label("")
    
    #=Los RV=#
    grid[5,2] = Label(string(RVA(table)))
    grid[5,3] = Label(string(RVB(table)))
    grid[5,4] = Label(string(RVAB(table)))
    grid[5,5] = Label("")
    grid[5,6] = Label("")
    grid[5,7] = Label("")
    
    setproperty!(grid, :column_homogeneous, true)
    
    window = Window()
    push!(window,grid)
    showall(window)
    
end

ANOVATable (generic function with 1 method)

## Ejemplo sobre como usar el programa.

Lo primero que hay que hacer es llamar a la función enterANOVAData.
Los parametros de esta son el número de filas (primer factor), el número de columnas (segundo factor), el tamaño de muestra
de cada uno de los bloques, los nombres de las categorias del primer factor (si son carreras serian "Actuaria, biología etc..."), los nombres de las categorias del segundo factor (digamos "realizar una actividad extra curricular", "no realizar act. extra"), y finalmente también se nos pide una tabla vacia que llenar. 

Una vez llamada la función aparecera una hoja de datos que llenar. cuando se apriete el botón "generate array" la tabla que dimos como argumento a la función se llenara con los datos que hayamos introducido. Corra el siguiente ejemplo.


In [149]:
#=Primero se llamar la funcion enter=#
rowNames = ["fila 1","fila 2"]
colNames = ["columna 1","columna2"]
rows = 2
cols = 2
sampleSize = 2
table = []
enterANOVAData(rows,cols,sampleSize,rowNames,colNames,table)


filled 2and 2
filled 2and 3
filled 3and 2
filled 3and 3


Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title=NULL, role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=-1, default-height=-1, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint=FALSE,

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
Array generated!


In [153]:
table

2-element Array{Any,1}:
 Any[Any[1.0, 2.0], Any[3.0, 4.0]]
 Any[Any[5.0, 6.0], Any[7.0, 8.0]]

Una vez que se ha llenado la table simplemente es cuestión de pasarle esta a la función ANOVATable

In [156]:
ANOVATable(table)

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title=NULL, role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=-1, default-height=-1, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint=FALSE,

In [ ]:
rowNames = ["Actuaría","Biología","Ciencias de la Computación","Física","Matemáticas"]
colNames = ["Realiza actividad extra","No realiza actividad extra"]
rows = 5
cols = 2
sampleSize = 20
table = []
enterANOVAData(rows,cols,sampleSize,rowNames,colNames)